### Задача ASR
Перед запуском кода на локальном компьютере создайте в корне три папки:
  - `texts_speech_recognition`
  - `waves_speech_recognition`
  - `waves_text_to_speech`

In [1]:
import speech_recognition as sr


# получение звука с микрофона
r = sr.Recognizer()
with sr.Microphone() as source:
  print('настройка микрофона на уровень окружающего шума..')
  r.adjust_for_ambient_noise(source)
  print('прослушивание..')
  # распознователь речи использует whisper
  try:
    # тайм-аут - это не максимальное время записи, а максимальное время ожидания начала воспроизведения звука
    audio = r.listen(source, timeout=5, phrase_time_limit=None)
    print('sending audio to Whisper..')
    outputString = r.recognize_whisper(audio_data=audio, model="medium", language="russian", show_dict=True)
    print(outputString["text"])
    # запись текста в файл
    with open(f"./texts_speech_recognition/microphone-results.txt", mode="w+", encoding="utf-8") as file_txt:
                        file_txt.write(outputString["text"])
    # запись аудио в WAV file
    with open("waves_speech_recognition/microphone-results.wav", "wb") as f:
      f.write(audio.get_wav_data())
  except sr.WaitTimeoutError:
    print('Время ожидания истекло')
  except sr.UnknownValueError:
    print("Whisper не удалось разобрать звук")
  except sr.RequestError as e:
    print(f"Не удалось запросить результаты у Whisper; {e}")

настройка микрофона на уровень окружающего шума..
прослушивание..
sending audio to Whisper..
 Привет, дружище! Рад знакомству! Как дела?


### Задача TTS

In [2]:
from transformers import VitsModel, AutoTokenizer, set_seed
import torch
import scipy
from ruaccent import RUAccent
from IPython.display import Audio
import warnings
warnings.filterwarnings("ignore")

device = 'cpu' #  'cpu' or 'cuda'

speaker = 0 # 0-woman, 1-man  

set_seed(555)  # make deterministic

# выбираем модель tts
model_name = "utrobinmv/tts_ru_free_hf_vits_low_multispeaker"
# инициализируем модель и токенизатор
model_path = f"./models/{model_name}"
tokenizer_path = f"./tokenizers/{model_name}"
model = VitsModel.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model.eval()

# загрузим акцентайзер
accentizer = RUAccent()
accentizer.load(omograph_model_size='turbo', use_dictionary=True, device=device)

# читаем распознанный текст
with open("./texts_speech_recognition/microphone-results.txt", mode="r", encoding="utf-8") as t:
  text = t.read()
# расставим акценты в тексте
text = accentizer.process_all(text)
print(text)
# выполним токенизацию
inputs = tokenizer(text, return_tensors="pt")
# выполним синтез речи
with torch.no_grad():
    output = model(**inputs.to(device), speaker_id=speaker).waveform
    output = output.detach().cpu().numpy()
# выполним сохранение в формат wav    
scipy.io.wavfile.write("./waves_text_to_speech/tts_results.wav", rate=model.config.sampling_rate,
                       data=output[0])
# выводим аудио
Audio(output, rate=model.config.sampling_rate)

прив+ет, друж+ище! р+ад знак+омству! к+ак дел+а?
